# Heuristic Functions
There are two heuristics:
- h1 - Heuristic using Manhattan distance
- h2 - Heuristic using number of misplaced tiles

0 represents a blank tile.

In [2]:
import numpy as np

In [3]:
def mat_to_dict(matrix):
    """
    Converts goal state to dictionary to get indices easier
    
    Parameters:
    matrix(np.ndarray): A 3x3 numpy array with each cell containing unique elements
    
    Returns:
    mat_dict(dict[int, tuple[int, int]]): A mapping of cell contents to a tuple
    contianing its indices
    """
    mat_dict = {}
    for i in range(matrix.shape[0]):
        row = matrix[i];
        for j in range(matrix.shape[1]):
            value = row[j];
            mat_dict[value] = (i, j)
    return mat_dict

In [4]:
def h1(curr_state, goal_dict):
    """
    Heuristic for calculating the distance of goal state using Manhattan distance
    
    Parameters:
    curr_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements
    goal_dict(dict[int, tuple[int, int]]): A mapping of cell contents to a tuple
    contianing its indices
    
    Returns:
    h(int): Heuristic value
    """
    h = 0
    for i in range(curr_state.shape[0]):
        for j in range(curr_state.shape[1]):
            value = curr_state[i][j]
            x = goal_dict[value][0]
            y = goal_dict[value][1]
            h += abs(i-x) + abs(j-y)
    return h

In [5]:
def h2(curr_state, goal_dict):
    """
    Heuristic for calculating the distance of goal state using number of misplaced tiles
    
    Parameters:
    curr_state(np.ndarray): A 3x3 numpy array with each cell containing unique elements
    goal_dict(dict[int, tuple[int, int]]): A mapping of cell contents to a tuple
    contianing its indices
    
    Returns:
    h(int): Heuristic value
    """
    h = 0
    for i in range(curr_state.shape[0]):
        for j in range(curr_state.shape[1]):
            value = curr_state[i][j]
            x = goal_dict[value][0]
            y = goal_dict[value][1]
            if ((x, y) != (i, j)):
                h += 1
    return h

In [6]:
GOAL_STATE = np.array([[1, 2, 3], [8, 0, 4], [7, 6, 5]])
print("Goal State:\n", GOAL_STATE)

Goal State:
 [[1 2 3]
 [8 0 4]
 [7 6 5]]


In [7]:
goal_dict = mat_to_dict(GOAL_STATE)
print("Goal dict:", goal_dict)

Goal dict: {1: (0, 0), 2: (0, 1), 3: (0, 2), 8: (1, 0), 0: (1, 1), 4: (1, 2), 7: (2, 0), 6: (2, 1), 5: (2, 2)}


In [8]:
# Change this to test
CURR_STATE = np.array([[2, 3, 0], [1, 8, 4], [7, 6, 5]])
print("Current State:\n", CURR_STATE)

Current State:
 [[2 3 0]
 [1 8 4]
 [7 6 5]]


In [9]:
# Heuristic 1
print("Value from h1:", h1(CURR_STATE, goal_dict))

Value from h1: 6


In [10]:
# Heuristic 2
print("Value from h2:", h2(CURR_STATE, goal_dict))

Value from h2: 5
